## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "default" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-44-202-5-220.compute-1.amazonaws.com:5000'


In [3]:
mlflow.search_experiments() # list_experiments API has been removed, you can use search_experiments instead.()

[<Experiment: artifact_location='s3://mlflow-remote-artifacts-100/1', creation_time=1748243825555, experiment_id='1', last_update_time=1748243825555, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-remote-artifacts-100/0', creation_time=1748243690946, experiment_id='0', last_update_time=1748243690946, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/26 14:21:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-remote-artifacts-100/1/5ebf05af2ace441b86dadcde335a36c2/artifacts'
🏃 View run resilient-crane-610 at: http://ec2-44-202-5-220.compute-1.amazonaws.com:5000/#/experiments/1/runs/5ebf05af2ace441b86dadcde335a36c2
🧪 View experiment at: http://ec2-44-202-5-220.compute-1.amazonaws.com:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-remote-artifacts-100/1', creation_time=1748243825555, experiment_id='1', last_update_time=1748243825555, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-remote-artifacts-100/0', creation_time=1748243690946, experiment_id='0', last_update_time=1748243690946, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [9]:
client.search_registered_models()

[]

In [13]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/26 14:22:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748244156961, current_stage='None', description='', last_updated_timestamp=1748244156961, name='iris-classifier', run_id='5ebf05af2ace441b86dadcde335a36c2', run_link='', source='s3://mlflow-remote-artifacts-100/1/5ebf05af2ace441b86dadcde335a36c2/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>